<a href="https://colab.research.google.com/github/kozltv/pMA_stabilnost/blob/main/pMA_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 0: Installation | Import

## 0. Install libraries (once)

## 1. Import libraries

In [1]:
import numpy as np
import os
import mne
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import auc
import warnings
warnings.filterwarnings('ignore')
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gsp
import gspread
from ipywidgets import widgets  
from ipywidgets import interact, interactive, fixed, interact_manual 

%matplotlib widget 
#%matplotlib notebook
#%matplotlib osx

## 2. Run functions

In [2]:
# To work with google sheets
#key_path = '/Users/ksenia.kozlova/Documents/GitHub/pma_grant'

def find_movement(data, time, mscl, window, thrs, thrs2, positive):
    '''Find movement

    arguments: 
      raw: DataFrame, filtered emg
      time: list with [start, end], contains start and end of raw data to analise
      mscl: str, name of EMG chanel
      window: int, window size in ms 
      thrs: int, threshold in uV to find movement start in the whole signal
      thrs2:  int, threshold in uV to find more accurate movement start/end moment in the window 
          where movement started/ended based in thrs
      positive: float, percentage of ticks in the window which should be more or eaqual
          to thrs/thrs2
    '''
    
    # 0. Select data
    # ms to select time window to find movement start (ms)
    try:
        start = time[0]
        end=time[1]
    except:
        start = time[0]
        end = raw.to_data_frame().time.max()
        
    dat = data[(data.time>=start)&(data.time<=end)]
            
    # NEW 1. Sliding window to check condition 
    y = abs(dat[mscl]).reset_index(drop=True)
    w_ms = window
    w_ticks = w_ms * 5

    a = y >= thrs

    s = np.zeros(len(a)-w_ticks+1, dtype=int)

    s[0] = np.sum(a[:w_ticks])

    for i in tqdm(range(1, len(s), 1)):
        s[i] = s[i-1] - a[i-1] + a[i+w_ticks-1]

    cond_result = s >= (w_ticks * positive)
    time = dat.time[:len(dat)-w_ticks+1]
    
      

    # 2. Find "start" and "end" of movement
    result3 = pd.DataFrame({'times':time, 'result':cond_result})
    result3['previous'] = result3.result.shift(1)
    result3['change'] = result3.result - result3.previous

    trig = result3[result3.change.isin([1,-1])].reset_index(drop=True)
    trig['movement'] = np.where(trig.change==1, 'start', 'end')



    # 3. select times in the suitable windows preciselly 
    res = []
    for tr in trig.times:
        wnd = np.abs(dat[(dat.time>=tr)&(dat.time<=tr+window)])

        if trig[trig.times==tr].movement.values[0] == 'start':
            # select time when uV achived thrs first
            first = wnd[wnd[mscl]>=thrs2].time.min()
            res.append([first,trig[trig.times==tr].movement.values[0]])
        else: 
            last = wnd[wnd[mscl]>=thrs2].time.max()
            res.append([last,trig[trig.times==tr].movement.values[0]])
        
    markup = pd.DataFrame(res)
    if markup.shape[1]>1:
        markup.columns = ['time','name']
    else:
        pass
        
    return markup, dat

def plot_signal(data, movement_time, mscl):
    """
    Interactive plot to see movement start closer

    Arguments:
      data: DataFrame with EMG by channels
      movement_time: array with times of movement start/end
    """

    fig, (ax0) = plt.subplots(nrows=1)
    for r in movement_time:
        ax0.axvline(x=r, color='r')

    ax0.plot(data.time, data[mscl], label='signal')
    ax0.legend()
    
    
def plot_emg_events(raw, movement_time, n_channels=1):
    """
    MNE Interactive plot to see whole EMG with movement time markups

    Arguments:
      raw: emg file
      movement_time: array with times of movement start/end
    """
    # create event list from movemet_times and name each event with order number
    events = pd.DataFrame({'time':np.array(movement_time)*5, 'event_value':1, 'event_name':list(range(1,len(np.array(movement_time))+1))})
    filt = np.array([50, 100, 150, 200])
    notch = raw.notch_filter(freqs=filt, verbose=False) #filter
    notch.plot(n_channels=n_channels, event_color='red', 
             events=np.array(events), 
             scalings=1e-3, 
             highpass=20, lowpass=1000)
    
def create_events(movement_time):
    """
    Creates event list from movemet_times and name each event with order number

    Arguments:
      movement_time: array with times of movement start/end
    """
    events = pd.DataFrame({'time':np.array(movement_time), 'event_value':1, 'event_name':list(range(1,len(np.array(movement_time))+1))})
    return events

    



def adjust_movements(ev):
    """
    Interactive program based on widgets to Manually adjast Algo movement markups

    Arguments:
      ev: array with time of movement markups which are need to be adjasted
    """
    
    #raw file to dataframe
    data = raw.filter(l_freq=20, h_freq=1000, verbose=False).to_data_frame()
    
    # create widget to list events with button 'next event'
    button=widgets.Button(
        value=False,
        description='Next event',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Description',
        icon='Saved' # (FontAwesome names without the `fa-` prefix)
                        )

    
    l = [] # for loop - indexes list of redo events 
    new_ts = [] # list to save new times stamp

    def plus(sender):
        l.append(1)
        index = np.array(l).sum()
        try:
            
            # create windget to Change event position
            w = widgets.FloatText(
            value=ev[index-1],
            min=ev[index-1]-800,
            max=ev[index-1]+3000,
            step=5,
            description='New Value uV:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            )
            
            # function for visualisation of new timestamp based on windget to Change event position
            def f(x):
                fig, (ax0) = plt.subplots(nrows=1)
                ax0.axvline(x=x, color='r')
                ax0.plot(data[(data.time>=ev[index-1]-800)&(data.time<=ev[index-1]+3000)].time, 
                         data[(data.time>=ev[index-1]-800)&(data.time<=ev[index-1]+3000)][mscl], label='signal') 

            
            # create Button to save new position
            b=widgets.Button(
                value=False,
                description='Save timestamp',
                disabled=False,
                button_style='', # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Description',
                icon='Saved' # (FontAwesome names without the `fa-` prefix)
                )
            
            
            saved_ts=[] # new timestamp list
            
            # function of button 'save timestemp' - it saves new value to the list
            def but(sender):
                saved_ts.append(w.value)

            b.on_click(but)
            display(b,interactive(f, x=w))
            new_ts.append(saved_ts)
            
        except Exception as e:
            print("You finished")


    button.on_click(plus)
   

    
    return display(button),new_ts


# 1. SET UP GOOGLE SHEET API
# ----------- CHANGE CREADENTIALS LOCATION ------------------------

# #current_dir = '/content'
# current_dir = key_path
# GSPREAD_KEYS_PATH = current_dir + '/esoteric-pad-326113-4ab241befdd8.json' # this is the directory where your file with key located
 

# # Some functions for authorizing (no need to change something here)
# def authorize_at_gspread(gspread_key_path=GSPREAD_KEYS_PATH):
#     """
#     Return authorized gspread object
#     :param gspread_key_path: path to credentials key
#     :return: gc object
#     """
#     scope = ['https://spreadsheets.google.com/feeds']
#     credentials = ServiceAccountCredentials.from_json_keyfile_name(GSPREAD_KEYS_PATH, scope)
#     gc = gspread.authorize(credentials)
#     return gc
# # authorization
# authorize_at_gspread()


# 2. Function for particular sheet connection
# def get_workbook(workbook_url, gspread_key_path=GSPREAD_KEYS_PATH):
#     """ Returns GSPREAD object with opened workbook.
#     Just open worksheet by referring to .worksheet(name) or use gspread_dataframe to load dataframe.
#     """
#     gc = authorize_at_gspread(gspread_key_path=gspread_key_path)
#     return gc.open_by_url(workbook_url)


# Damir - to write to a sheet fucntion
# def tospreadsheet (df, workbook_link, header = 'Title', key = 'new'):
#     """Import dataframe to google spreadsheet by link
#     Args: 
#         df: imported dataframe
#         workbook: link to dataframe
#         header: header of worksheet 
#         key: default 'new'
#         if you want to use existing worksheet pass 'to existing sheet'

#     """
#     if key == 'new':
#         workbook = get_workbook(workbook_link)
#         wks = workbook.add_worksheet(header, len(df.index),len(df.columns))
#         gsp.set_with_dataframe(wks, df, row = 1, col =1, include_column_header=True)

#     elif key == 'existing':
#         workbook = get_workbook(workbook)
#         wks = workbook.worksheet(header)
#         gsp.set_with_dataframe(wks, df, row = 1, col =1, include_column_header=True)
        

def gsheet(header,new_t,bad,slow, sheet=False):
    
    new = pd.DataFrame({'events_to_correct':redo})
    new2=pd.concat([new, pd.DataFrame(new_t[1])], axis=1).set_index('events_to_correct')

    result = movement_time[['name','time']]
    result['event_name'] = result.reset_index()['index']+1
    result = result.set_index('event_name')

    final = pd.concat([result,new2],axis=1)
    final['muscle'] = mscl
    final['file'] = path[-20:path.find('.vhdr')][path[-20:path.find('.vhdr')].find('/')+1:]
    #final_corrected_movement_time = last filled column before "muscle" column
    final['final_corrected_movement_time'] = final.apply(lambda x: x.iloc[(x.dropna().to_frame().transpose().columns.get_loc('muscle'))-1],axis=1)

    #add Bad and Slow lables
    final['slow'] = pd.DataFrame({'index':slow, 'Slow':True}).set_index('index')
    final['bad'] = pd.DataFrame({'index':bad, 'Bad':True}).set_index('index')
    
    link = 'https://docs.google.com/spreadsheets/d/1g0wJeyONCuivtppNc352MBkBIyOgY7FxkQq5XgaBjsg/edit?usp=sharing'

    if sheet == True:
        pass
        #tospreadsheet(final, link, header)
    elif sheet == False:
        file = header + '.xls'
        final.to_excel(file)
        print('Movement data saved to file ', file)

    return final


#PMA functions

def envelop(time, mscl, data, sub_window_size):
    """
    time: list, time where calculate envelop
    mscl: str
    data: df
    subwindow_size: int, ms for envelop
    """
    
    start = time[0]
    end = time[1]    
    array = np.array(data[mscl])
    
    start_index = data[data.time>=start].index.min()
    end_index = data[data.time<=end].index.max()    
        
        
    sub_windows = (
        start_index + 
        # *5 cos 5k HZ
        np.expand_dims(np.arange(sub_window_size*5), 0) +
        # +2 to not mess last 2 indexes; subtraction of sub_window_size to avoid extra indexes missed in data array
        np.expand_dims(np.arange(end_index - start_index - sub_window_size*5 + 2), 0).T
        )
    
    envelop = pd.DataFrame(array[sub_windows]).apply(lambda x: (abs(x)).max(), axis = 1)
    
    return envelop

def pma_find_preact_thrs(sheet, data, preactivation_time, mscl):
    """
    sheet: df, data about movement from sheet with correected time
    data: df, raw emg data
    preactivation_time: int, is ms for preactivation window analisys
    mscl: str, name of channel
    """
    # extract movement start corrected
    movement_time_cor = sheet[sheet.name=='start'].final_corrected_movement_time
    windows = data[data.time.isin(movement_time_cor)].index
    each_five = list(range(0,len(windows),5)) #select each 1st index among 5 indexes with te same time (5000 hz) 
    # find inndex for movement
    movemenet_start_index = windows[each_five]
    #get trial and its slow lable
    slow_trial_list = sheet[['trial_number','slow']].drop_duplicates().slow
    #save all inffor to df
    preactivation_boards_list = pd.DataFrame({'start_time':np.array(data.time)[movemenet_start_index-preactivation_time*5],
                         'end_time':np.array(data.time)[movemenet_start_index],
                        'Trial number':sheet.trial_number.unique(),
                        'Is slow':slow_trial_list})
    
    thrss = []
    # go through [start, end] times in preactivation
    for preact_time in np.array(preactivation_boards_list):
        env = envelop(preact_time, mscl, data, 50)
        thrs = env.mean() + env.std()*2
        
        # saves thr, original trial number and slow lable
        thrss.append([thrs, preact_time[2], preact_time[3] ])
    
    thrss = pd.DataFrame(thrss)
    thrss[0] = thrss.fillna(np.mean(thrss))
    return thrss
        
def get_movement_indexes(sheet, name, df):
    """
    sheet: df from sheet with 'final_corrected_movement_time' column
    name: str, time name from sheet start/end of movement
    df: raw.to_data_frame()
    """
    movement_time_cor = sheet[sheet.name==name].final_corrected_movement_time
    windows = df[df.time.isin(movement_time_cor)].index
    each_five = list(range(0,len(windows),5)) #select each 1st index among 5 indexes with te same time (5000 hz) 
    movemenet_index = windows[each_five]
    return movemenet_index

def find_pma(data, mscl, thrss, pma_start_positive):
    
    """
    data: df of emg 
    mscl: str, name of channel like 'lADM' 
    thrss: list, preactivation thresholds list for the mscl
    pma_start_positive: float, share of picks whihch should be >= preact. thrs
    """

    ind_end = get_movement_indexes(sheet, 'end', data)
    ind_st = get_movement_indexes(sheet, 'start', data)

    all_ind = np.stack((ind_st,ind_end), axis=1)

    window_indexes = []
    
    for r in all_ind:
        window_indexes.append(np.array(range(r[0],r[1]+1)))
    
    movement_indexes = np.asarray(window_indexes)

    post_windows = pd.DataFrame()
    
    post_windows['indexes'] = movement_indexes
    post_windows['Trial in session'] = thrss[1]
    post_windows['Is slow'] = thrss[2]
    
    post_windows['preact_window'] = post_windows.apply(lambda x: np.array(data[mscl][np.array(range(x.indexes.min()-5000, x.indexes.min()+1))[np.array(range(x.indexes.min()-5000, x.indexes.min()+1))>=0]]), axis = 1)
    post_windows['signal'] = post_windows.apply(lambda x: np.array(data[mscl][x.indexes]), axis=1)
    post_windows['time'] = post_windows.apply(lambda x: np.array(data.time[x.indexes]), axis=1)
    post_windows['preactivation_mean_uv'] = post_windows.apply(lambda row: np.mean(abs(row.preact_window)), axis=1)
    post_windows['preactivation_max_uv'] = post_windows.apply(lambda row: np.max(abs(row.preact_window)), axis=1)
    post_windows['preactivation_std_uv'] = post_windows.apply(lambda row: np.std(abs(row.preact_window)), axis=1)
    post_windows['preactivation_auc_uv'] = post_windows.apply(lambda row: auc(range(len(row.preact_window)),abs(row.preact_window)), axis=1)
        

    post_windows['movement_start'] = post_windows.apply(lambda row: row['time'].min(), axis = 1)
    post_windows['movement_end'] = post_windows.apply(lambda row: row['time'].max(), axis = 1)
    post_windows['preac_thrs'] = thrss[0]
    
    
    def find_start_pma(y, tr, pma_start_positive):
        w = 50 #len of sliding window in ticks (10 ms X 5 hz)
        w_tr = int(pma_start_positive*w) #number of ticks over tr to find pma
        for i in range (len(y)-w):
            y2 = [x for x in y[i:i+w] if abs(x)>tr]
            #y[i:i+w][y[i:i+w]>tr]
            if len(y2) >= w_tr:
                #print(y2,i)
                return i

    def find_end_pma(y, tr, pma_start_positive):
        last_index = find_start_pma(y[::-1], tr, pma_start_positive)
        return len(y)-last_index
        

    # based on preactivation
    post_windows['pma_start_index'] = post_windows.apply(lambda row: find_start_pma(row.signal, row.preac_thrs, pma_start_positive), axis=1)
    post_windows['pma_end_index'] = post_windows.apply(lambda row: pd.DataFrame(row.signal)[abs(row.signal)>row.preac_thrs].index.max()
                                                  , axis=1)
#     post_windows['pma_end_index'] = post_windows.apply(lambda row: find_end_pma(row.signal, row.preac_thrs, pma_start_positive), axis=1)

    post_windows['pma_end_index'] = np.where(post_windows['pma_start_index'].isna()==False, post_windows['pma_end_index'], np.NaN) 
    

    def find_time(array_with_data, index):
        try:
            return int(array_with_data[int(index)])
        except Exception as s:
            pass


    post_windows['pma_time_start'] = post_windows.apply(lambda row: find_time(row.time, row.pma_start_index)
                                                  , axis=1)
    post_windows['pma_time_end'] = post_windows.apply(lambda row: find_time(row.time, row.pma_end_index)
                                                  , axis=1)
    post_windows['pma_duration_ms'] = post_windows['pma_time_end'] - post_windows['pma_time_start']
    
    post_windows['muscle'] = mscl

    
    return post_windows


def check_pma(pma_df, pma_duration):
    """
    pma_df: df recieved from find_pma function
    pma_duration: int, ms how long pma should be
    
    """
    check_pma_df = pma_df
    check_pma_df['pma_check'] = check_pma_df.dropna().apply(lambda x: x.signal[int(x.pma_start_index):int(x.pma_end_index+1)], axis = 1)
    check_pma_df['rms_pma'] = check_pma_df.dropna().apply(lambda x: ((abs(x.pma_check)**2).mean())**0.5, axis = 1)

    def env_pma(line, sub_window_size):
        array = line.signal
        start_index = int(line.pma_start_index)
        end_index = int(line.pma_end_index)   

        sub_windows = (
            start_index + 
            # *5 cos 5k HZ
            np.expand_dims(np.arange(sub_window_size*5), 0) +
            # +2 to not mess last 2 indexes; subtraction of sub_window_size to avoid extra indexes missed in data array
            np.expand_dims(np.arange(end_index - start_index - sub_window_size*5 + 2), 0).T
            )

        envelop = pd.DataFrame(array[sub_windows]).apply(lambda x: (abs(x)).max(), axis = 1)

        return np.array(envelop).mean()

    check_pma_df['pma_envelop_mean'] =  check_pma_df.dropna().apply(lambda x: env_pma(x,pma_duration), axis=1 )

    return check_pma_df


def rms_pma(pma_df):
    """
    pma_df: df recieved from find_pma function
    """
    
    pma_df2 = pma_df.reset_index(drop=True)
    
    # "try" for cases when no pma 
    try:
        pma_df2['pma_wnd'] = pma_df2[['signal','pma_start_index','pma_end_index']].dropna().apply(lambda x: x.signal[int(x.pma_start_index):int(x.pma_end_index+1)], axis = 1)
        pma_df2['pma_mean_uv'] = pma_df2[['pma_wnd']].dropna().apply(lambda row: np.mean(abs(row.pma_wnd)), axis=1)
        pma_df2['pma_max_uv'] = pma_df2[['pma_wnd']].dropna().apply(lambda row: np.max(abs(row.pma_wnd)), axis=1)
        pma_df2['pma_std_uv'] = pma_df2[['pma_wnd']].dropna().apply(lambda row: np.std(abs(row.pma_wnd)), axis=1)
        pma_df2['pma_auc_uv'] = pma_df2[['pma_wnd']].dropna().apply(lambda row: auc(range(len(row.pma_wnd)),abs(row.pma_wnd)), axis=1)
    except:
        pass
    
    def rms(line, sub_window_size):
        array = line
        start_index = 0
        end_index = len(array) - 1  

        sub_windows = (
            start_index + 
            # *5 cos 5k HZ
            np.expand_dims(np.arange(sub_window_size*5), 0) +
            # +2 to not mess last 2 indexes; subtraction of sub_window_size to avoid extra indexes missed in data array
            np.expand_dims(np.arange(end_index - start_index - sub_window_size*5 + 2), 0).T
            )

        #rms_pma = pd.DataFrame(array[sub_windows]).apply(lambda x: ((abs(x)**2).mean())**0.5, axis = 1)
        rms_pma = np.array([((abs(wnd)**2).mean())**0.5 for wnd in array[sub_windows]]).mean()

        return rms_pma
    
    # "try" for cases when no pma 
    try:
        pma_df2['pma_rms_mean'] =  pma_df2[['pma_wnd']].dropna().apply(lambda x: rms(x.pma_wnd, 30), axis=1 )
    except:
        pass
    pma_df2['preactivation_window_rms_mean'] =  pma_df2[['preact_window']].apply(lambda x: rms(x.preact_window, 30), axis=1 )
     
    return pma_df2


def active_hand(all_pma_data, data, rms_time, active_mscl):
    
    """
    
    pma_data: df given from pma function columns indexes, movement_start/end are needed
    data: df, emg data freq. filtered - to get pre/activation data in right hand
    rms_time: ms, window size to calc rms
    active_mscl: str, active muscle
    """
    
    all_pma_data = all_pma_data.reset_index(drop=True)
    active_hand = pd.DataFrame()
    
    active_hand['indexes'] = all_pma_data.indexes
    active_hand['Trial in session'] = all_pma_data['Trial in session']
    active_hand['Is slow'] = all_pma_data['Is slow']
    active_hand['movement_start'] = all_pma_data.movement_start
    active_hand['movement_end'] = all_pma_data.movement_end
    
    def rms(line, sub_window_size):
        array = line
        start_index = 0
        end_index = len(array) - 1  

        sub_windows = (
            start_index + 
            # *5 cos 5k HZ
            np.expand_dims(np.arange(sub_window_size*5), 0) +
            # +2 to not mess last 2 indexes; subtraction of sub_window_size to avoid extra indexes missed in data array
            np.expand_dims(np.arange(end_index - start_index - sub_window_size*5 + 2), 0).T
            )
        #rms_pma = np.array([((abs(wnd)**2).mean())**0.5 for wnd in array[sub_windows]]).mean()
        rms_pma = pd.DataFrame(array[sub_windows]).apply(lambda x: ((abs(x)**2).mean())**0.5, axis = 1).mean()
        return rms_pma

    active_hand_all = pd.DataFrame()
    for m in tqdm(['rAPB','rADM','rECU']): 
        active_hand['preactivation_window'] = active_hand.apply(lambda x: np.array(data[m][np.array(range(x.indexes.min()-5000, x.indexes.min()+1))[np.array(range(x.indexes.min()-5000, x.indexes.min()+1))>=0]]), axis = 1)
        active_hand['activation_window'] = active_hand.apply(lambda x: np.array(data[m][x.indexes]), axis=1)

        active_hand['preactivation_mean_uv'] = active_hand.apply(lambda row: np.mean(abs(row.preactivation_window)), axis=1)
        active_hand['preactivation_max_uv'] = active_hand.apply(lambda row: np.max(abs(row.preactivation_window)), axis=1)
        active_hand['preactivation_std_uv'] = active_hand.apply(lambda row: np.std(abs(row.preactivation_window)), axis=1)
        active_hand['preactivation_auc_uv'] = active_hand.apply(lambda row: auc(range(len(row.preactivation_window)),abs(row.preactivation_window)), axis=1)
        #active_hand['preactivation_window_baseline_rms_mean'] = active_hand.apply(lambda row: rms(abs(row.preactivation_window),rms_time), axis=1)

        active_hand['activation_mean_uv'] = active_hand.apply(lambda row: np.mean(abs(row.activation_window)), axis=1)
        active_hand['activation_max_uv'] = active_hand.apply(lambda row: np.max(abs(row.activation_window)), axis=1)
        active_hand['activation_std_uv'] = active_hand.apply(lambda row: np.std(abs(row.activation_window)), axis=1)
        active_hand['activation_auc_uv'] = active_hand.apply(lambda row: auc(range(len(row.activation_window)),abs(row.activation_window)), axis=1)
        #active_hand['activation_window_baseline_rms_mean'] = active_hand.apply(lambda row: rms(abs(row.activation_window),rms_time), axis=1)
        active_hand['muscle'] = m
        active_hand_all = active_hand_all.append(active_hand)
        
    active_hand_all['movement_start'] = np.where(active_hand_all['muscle'] == active_mscl, active_hand_all['movement_start'], np.NaN)
    active_hand_all['movement_end'] = np.where(active_hand_all['muscle'] == active_mscl, active_hand_all['movement_end'], np.NaN)
    active_hand_all['active_muscle'] = active_mscl
    
    
    return active_hand_all


# <br><br> Part 1: find Movement onset

## 0. Uppload data + data filtering

### Adjast: Insert subject .vhdr file PATH

In [155]:
path = '/Users/ksenia.kozlova/Documents/GitHub/pma_grant/EMG data/KoYa_30/KoYa_30_70max_rADM_1.vhdr'




### LOAD DATA
emg = mne.io.read_raw_brainvision(path,verbose=False)

# FILTERS
# high pass and low pass filters
filters_hp_lp = emg.load_data().filter(l_freq=20, h_freq=1000, verbose=False)
# notch filters
filters_notch = np.array([50, 100, 150, 200])

raw = filters_hp_lp.notch_filter(freqs=filters_notch, verbose=False)
data = raw.to_data_frame()

<p style="color:#FFA500";><b>Information about the record</p>


In [156]:
print(raw.info)

<Info | 7 non-empty values
 bads: []
 ch_names: rAPB, rADM, rECU, lAPB, lADM, lECU
 chs: 6 EEG
 custom_ref_applied: False
 highpass: 20.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2021-07-28 22:31:19 UTC
 nchan: 6
 projs: []
 sfreq: 5000.0 Hz
>


### Plot EMG

<p style="color:#FFA500";><b>Check whether record is ok</p>

In [158]:
try:
    raw.plot(verbose=False, n_channels = 1)
except:
    print('Error')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [154]:
raw.info

<Info | 7 non-empty values
 bads: []
 ch_names: rAPB, rADM, rECU, lAPB, lADM, lECU
 chs: 6 EEG
 custom_ref_applied: False
 highpass: 20.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2021-07-29 13:03:09 UTC
 nchan: 6
 projs: []
 sfreq: 5000.0 Hz
>

## 1. Find movement with ALGO

### 1. Setup AlGO parameters

### Select trail part of data with 3-5 movements 

#### Adjast: muscle, start_trial, end_trial

In [165]:
# active muscle channel
muscle = 'lADM'
# time in ms
start_trial = 86000
end_trial = 96000


# no changes
trial_data = data[(data.time>=start_trial)&(data.time<=end_trial)]

Plot Trial signal
<p style="color:#FFA500";><b>check trial data</p>

In [166]:
# no change
fig, (ax0) = plt.subplots(nrows=1)
ax0.plot(trial_data.time, abs(trial_data[str(muscle)]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Adjast. parameters (thrs, thrs2) !_

In [122]:
# ADJUST

# 1. Thresholds, (absolut value) uV activity threshold which concidered as a movement 
# thrs - general, thrs2 is more accurate in subwindow 
thrs = 200
thrs2 = 60

# 2. time window to analyse (ms)
# 650 is optimal 
window = 655

# 3. percentage of picks which should be higher than thrs in the window 
# 0.05 is optimal
positive = 0.05

### Try algo settings on trial data at first.

In [123]:
# no changes
# start and end in ms
trial_time = [start_trial, end_trial]
# Run algo on trial data
trial_movement_time, trial_data = find_movement(data, trial_time, muscle, window, thrs, thrs2, positive)

trial_movement_time

,time,name
0,6938.0,start
1,10499.0,end
2,12600.0,start
3,16806.0,end
4,19558.0,start
5,23481.0,end


<p style="color:#FFA500";><b>check how algo works</p>

In [124]:
# plot trial
plot_signal(trial_data, trial_movement_time.time, muscle)
#plt.hlines(50, xmin=5000, xmax=22500,color='r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

__If the Movement start is not acurate enough, Adjast thrs,thrs2 and try again #2__

### 2. Apply to the whole recording

In [125]:
last = data.time.max()

__here time = [0] which means "start from 0 second"__

In [130]:
movement_time, movement_data = find_movement(data, [0], muscle, window, thrs, thrs2, positive)

#### check how algo works for whole file

In [131]:
# plot Algo Markup way1
plot_signal(movement_data, movement_time.time, muscle)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2. Manual adjastment of Algo Movement Markup

<p style="color:#FFA500";><b>check how algo works for whole file</p>

In [140]:
# plot Algo Markup way2
plot_emg_events(raw, movement_time.time,6)

Setting up band-pass filter from 20 - 1e+03 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 20.00, 1000.00 Hz: -6.02, -6.02 dB



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 1. Adjast: Fill events start numbers to redo/slow/bad !

In [ ]:
start_bad_number = 47
start_good_number = 55
print(list(range(start_bad_number,start_good_number,2)))

In [136]:
## change

#list to manually redo
redo = [1]
#list with slow (second spike is later than 200 ms)
slow = [1,11,13,15,17,19,21]

#list with Bad - to delete
bad = [25,33,45,49]


In [137]:
# no change
events = create_events(movement_time.time)

In [138]:
events[events.event_name.isin(redo)].reset_index().time

0    471.0
Name: time, dtype: float64

### 2. Manually adjast "Redo" movements

In [139]:
mscl = muscle
new_timings = adjust_movements(events[events.event_name.isin(redo)].reset_index().time)

Button(description='Next event', icon='Saved', style=ButtonStyle(), tooltip='Description')

Button(description='Save timestamp', icon='Saved', style=ButtonStyle(), tooltip='Description')

interactive(children=(FloatText(value=471.0, description='New Value uV:', step=5.0), Output()), _dom_classes=(…

__Plot signal with final movement start Markups__

## 3. Save adjasted Movement times to Excel

##### Adjast: run cell and fill "Movement file name"

In [ ]:
print('Current emg file: ', path)
file_name = input('Movement file name (subject + emg file name): ')

corrected_time = gsheet(file_name , new_timings, bad, slow)

<p style="color:#FFA500";><b>check final table</p>

In [ ]:
corrected_time

<p style="color:#FFA500";><b>final check how movement was marked up</p>

In [ ]:
plot_emg_events(raw, corrected_time.final_corrected_movement_time)

 # <br><br><br><br>  Part 2A. pMA analysis (Session level)

## 0. Get data + data filtering

#### Adjast: Set up your sign_path

In [3]:
# MAC OS = '/' WINDOWS = '\\'
SIGN_PATH = '/'

#### Adjast: run cell and insert .vhrd path and .xlx path

In [4]:
try:
    subj = input(f'1. Get EMG data (.vhdr) \n\n Continue with file {path} ? \n Y/N: ')
except:
    subj = 'N'


if subj == 'Y':
    try:
        subject_path = path
    except:
        pass     
else:
    subject_path = input('Insert subject .vhdr path: ')

### LOAD DATA
# FILTERS
raw = mne.io.read_raw_brainvision(subject_path, verbose=False)
# high pass and low pass filters
filters_hp_lp = raw.load_data().filter(l_freq=20, h_freq=1000, verbose=False)
# notch filters
filters_notch = np.array([50, 100, 150, 200])
raw_filtered = filters_hp_lp.notch_filter(freqs=filters_notch, verbose=False)
pma_data = raw_filtered.to_data_frame()
 

movement_path = input('2. Gett movement data from Excel \n Insert .xls file path with movement info: ')
sheet = pd.read_excel(movement_path)
sheet['slow'] = np.where((sheet['slow'] == '')|(sheet.slow.isna()==True), sheet['slow'].shift(1), sheet['slow'])
sheet['bad'] = np.where((sheet['bad'] == '')|(sheet.bad.isna()==True), sheet['bad'].shift(1), sheet['bad'])
sheet['trial_number'] = (2+ np.array(range(sheet.shape[0])))//2
sheet = sheet[(sheet.bad!='TRUE')&(sheet.bad!=1)]
print('Your file: ')
    
sheet.head(3)

Insert subject .vhdr path: /Users/ksenia.kozlova/Documents/GitHub/pma_grant/EMG data/DaSa_27/DaSa_27_70max_rADM_1.vhdr
2. Gett movement data from Excel 
 Insert .xls file path with movement info: /Users/ksenia.kozlova/Documents/GitHub/pma_grant/Movement data/DaSa_27/DaSa_27_70max_rADM_1.xls
Your file: 


,Unnamed: 0,name,time,0,muscle,file,final_corrected_movement_time,slow,bad,trial_number
0,1,start,8241,NaN,rADM,27_70max_rADM_1,8241,NaN,NaN,1
1,2,end,16800,NaN,rADM,27_70max_rADM_1,16800,NaN,NaN,1
2,3,start,22638,NaN,rADM,27_70max_rADM_1,22638,NaN,NaN,2


In [ ]:
/Users/ksenia.kozlova/Documents/GitHub/pma_grant/EMG data/ArtDz_9/pma_ADM_1.vhdr
/Users/ksenia.kozlova/Documents/GitHub/pma_grant/Movement data/ArtDz_9/ArtDZ_9_pma_APB_3.xls

<p style="color:#FFA500";><b>Check that record and movemet start comply</p>

__Plot EMG and movement start__

In [6]:
mscl = sheet.muscle.unique()
movement_time = sheet.final_corrected_movement_time
try:
    plot_emg_events(raw_filtered, movement_time,3)
except:
    plot_signal(pma_data, movement_time, mscl)

Setting up band-pass filter from 20 - 1e+03 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 20.00, 1000.00 Hz: -6.02, -6.02 dB



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

__PMA processing__

## 1. Analyse left hand

In [7]:
pma_all = pd.DataFrame()

PMA_START_POSITIVE = 0.25


for muscle in tqdm(['lAPB','lADM','lECU']):
    # culculate preactivation thrs in preactivatuon window based on envelop
    thrss = pma_find_preact_thrs(sheet, pma_data, 1000, muscle)
    # find pma times 
    pma = find_pma(pma_data, muscle, thrss, PMA_START_POSITIVE) 
    # calculate rms or each pma
    rms = rms_pma(pma)
    # save all muscles to one df
    pma_all=pma_all.append(rms)
    print(muscle, 'is done')
    
    
pma_all['pma_latency_ms'] = pma_all.pma_time_start - pma_all.movement_start

lAPB is done
lADM is done
lECU is done



In [8]:
pma_all['session'] = subject_path[-6]
loc = len(subject_path)-subject_path[::-1].find(SIGN_PATH)
pma_all['file'] = subject_path[loc:]
pma_all.head(2)

,indexes,Trial in session,Is slow,preact_window,signal,time,preactivation_mean_uv,preactivation_max_uv,preactivation_std_uv,preactivation_auc_uv,...,pma_wnd,pma_mean_uv,pma_max_uv,pma_std_uv,pma_auc_uv,pma_rms_mean,preactivation_window_rms_mean,pma_latency_ms,session,file
0,"[41203, 41204, 41205, 41206, 41207, 41208, 412...",1.0,NaN,"[0.5118861788031731, 1.2408663637754176, 1.900...","[0.6826842675688011, 1.762350982809292, 1.4260...","[8241, 8241, 8241, 8241, 8241, 8242, 8242, 824...",0.817781,3.758751,0.619651,4089.126723,...,"[-1.9975959728444588, -0.45943563507699686, -0...",1.169822,25.167845,1.413801,48306.992154,1.420504,1.020346,231.0,1,DaSa_27_70max_rADM_1.vhdr
1,"[113188, 113189, 113190, 113191, 113192, 11319...",2.0,NaN,"[-1.411050786895527, -0.07712989763244252, 1.1...","[-1.7408111764057033, -0.49101261328120505, 1....","[22638, 22638, 22638, 22638, 22638, 22639, 226...",0.869022,4.605709,0.651511,4344.402782,...,"[-2.37207594845316, -2.7504752042890983, -2.20...",1.164720,33.884582,1.531771,46402.395844,1.443970,1.077328,144.0,1,DaSa_27_70max_rADM_1.vhdr


### Check pma start by plot

#### Adjast: change mscl - muscle for visualization

In [9]:
mscl = 'lADM'
markups = ((pma_all[pma_all.muscle==mscl].pma_time_start).append(pma_all[pma_all.muscle==mscl].pma_time_end.append(pma_all[pma_all.muscle==mscl].movement_start.append(pma_all[pma_all.muscle==mscl].movement_end)))).sort_values().dropna()

try:
    plot_emg_events(raw_filtered, markups,1)
except:
    plot_signal(pma_data, markups, mscl)
    

Setting up band-pass filter from 20 - 1e+03 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 20.00, 1000.00 Hz: -6.02, -6.02 dB



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Check pma average numbers

In [11]:
pma_all.columns

Index(['indexes', 'Trial in session', 'Is slow', 'preact_window', 'signal',
       'time', 'preactivation_mean_uv', 'preactivation_max_uv',
       'preactivation_std_uv', 'preactivation_auc_uv', 'movement_start',
       'movement_end', 'preac_thrs', 'pma_start_index', 'pma_end_index',
       'pma_time_start', 'pma_time_end', 'pma_duration_ms', 'muscle',
       'pma_wnd', 'pma_mean_uv', 'pma_max_uv', 'pma_std_uv', 'pma_auc_uv',
       'pma_rms_mean', 'preactivation_window_rms_mean', 'pma_latency_ms',
       'session', 'file'],
      dtype='object')

In [16]:
for m in range(pma_all[pma_all.muscle.isin(['lAPB'])].shape[0]):
    i = m

    ax, pl = plt.subplots()
    # lAPB blue
    ax = plt.plot(abs(pma_all[pma_all.muscle.isin(['lAPB','lADM'])].preact_window[i].reset_index(drop=True)[0]))
    # lADM orange
    ax = plt.plot(abs(pma_all[pma_all.muscle.isin(['lAPB','lADM'])].preact_window[i].reset_index(drop=True)[1]))
    print('movement start:', pma_all[pma_all.muscle.isin(['lAPB','lADM'])].movement_start[i].reset_index(drop=True)[0],'\n',
          'preact thr:', pma_all[pma_all.muscle.isin(['lAPB','lADM'])].preac_thrs[i].reset_index(drop=True)[0],'\n',
        'mean uV lAPB:', round(pma_all[pma_all.muscle.isin(['lAPB','lADM'])].preactivation_mean_uv[i].reset_index(drop=True)[0],2),'\n',
             'mean uV lADM:', round(pma_all[pma_all.muscle.isin(['lAPB','lADM'])].preactivation_mean_uv[i].reset_index(drop=True)[1],2))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

movement start: 8241 
 preact thr: 3.8058239054274567 
 mean uV lAPB: 0.82 
 mean uV lADM: 0.94


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

movement start: 22638 
 preact thr: 4.195421884291594 
 mean uV lAPB: 0.87 
 mean uV lADM: 0.9


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

movement start: 195250 
 preact thr: 4.202626165921512 
 mean uV lAPB: 0.86 
 mean uV lADM: 2.13


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

movement start: 202711 
 preact thr: 4.736770529118516 
 mean uV lAPB: 0.94 
 mean uV lADM: 1.21


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

movement start: 209768 
 preact thr: 4.449088194943995 
 mean uV lAPB: 0.93 
 mean uV lADM: 1.32


In [ ]:
round(pma_all.groupby(['muscle']).agg({'pma_duration_ms':'mean',
                                 'pma_latency_ms':'mean',
                                 'pma_rms_mean':'mean',
                                    'preactivation_max_uv':'mean',
       'preactivation_mean_uv':'mean', 'preactivation_auc_uv':'mean',   
                                       'preac_thrs':'mean',
                                'pma_max_uv':'mean',
                                 'pma_mean_uv':'mean',
                                 'pma_wnd':'count', # = how many pma found
                                 'movement_start':'count' # = how many movemnts
                                }),1)

In [ ]:
pma_format = round(pma_all[[
       'preactivation_mean_uv',
       'preactivation_max_uv',
       'preactivation_std_uv',
       'preactivation_auc_uv',
       'preactivation_window_rms_mean',
        'preac_thrs','pma_duration_ms', 'pma_latency_ms',
       'pma_mean_uv', 'pma_max_uv', 'pma_std_uv','pma_auc_uv',
       'pma_rms_mean',
        'muscle','Trial in session', 'session', 'file']].set_index(['Trial in session','muscle','session', 'file']).unstack(1),2)

pma_format.head(4)

## 2. Analyse right hand

In [ ]:
active_muscle = sheet.muscle.unique()[0]
r_hand = active_hand(pma_all[pma_all.muscle=='lADM'][['movement_start', 'movement_end','indexes','Trial in session','Is slow']], pma_data, 10, active_muscle)

r_hand['session'] = subject_path[-6]
loc = len(subject_path)-subject_path[::-1].find(SIGN_PATH)
r_hand['file'] = subject_path[loc:]

r_hand.head(3)

### Check active hand average numbers

In [ ]:
round(r_hand.groupby(['muscle']).agg({'preactivation_mean_uv':'mean',
                                 'activation_mean_uv':'mean',
                                 'activation_auc_uv':'mean',
                                 'movement_start':'count' # = how many movements done
                                }),1)

In [ ]:
r_hand_format = round(r_hand[['movement_start','movement_end',
       'preactivation_mean_uv',
       'preactivation_max_uv',
       'preactivation_std_uv',
       'preactivation_auc_uv',
       'activation_mean_uv',
       'activation_max_uv',
        'activation_auc_uv',
       'activation_std_uv',
       'muscle','Trial in session','session', 'Is slow','file']].set_index(['Trial in session','muscle','session', 'file']).unstack(1),2)

r_hand_format.head(3)

## 3. Merge Data right and left hand

In [ ]:
pma_format.head(2)

In [ ]:
r_hand_format.head(2)

### Check final output Right and Left hands

In [ ]:
output = r_hand_format.merge(pma_format, how='left', on = ['Trial in session','file','session'], suffixes = ['_active','_pma'])
output['active_muscle']  = str(r_hand.active_muscle.unique()[0])
output = output.set_index('active_muscle', append=True)
output

## 4. Save pMA output to Excel

In [ ]:
print('Current EMG file for subject:',subject_path)
subject_name = input('Sibject code: ')
file_name = subject_name + '_' + output.reset_index().file[0] +'_s' + output.reset_index().session[0] + '.xls'
output.to_excel(file_name)
print('pma data saved to: ', file_name)

# <br><br><br>  Part 2B. pMA analysis (Subject level)

## 0. Get data

#### Adjast: Path for folders with movement excels and EMG files

In [ ]:
excel_files_path = '/Users/ksenia.kozlova/Documents/GitHub/pma_grant/Movement data'
emg_files_path = '/Users/ksenia.kozlova/Documents/GitHub/pma_grant/EMG data'

__just run__

In [ ]:
excel_files = np.array(os.listdir(excel_files_path))

mov_files_all = pd.DataFrame()
for subject in excel_files[excel_files!='.DS_Store']:
    df_mov_files = pd.DataFrame()
    df_mov_files['excel_file'] =  os.listdir(excel_files_path + '/'+ subject)

    df_mov_files['session'] = np.where(df_mov_files.excel_file.str.contains('1|2|3')==True, df_mov_files.excel_file.str[-5], np.NaN)
    df_mov_files['muscle_index'] = np.where((df_mov_files.excel_file.str.contains('APB')==True), df_mov_files.excel_file.str.find('APB'),
                                        df_mov_files.excel_file.str.find('ADM'))
    df_mov_files['muscle'] = df_mov_files.apply(lambda x: x.excel_file[x.muscle_index:x.muscle_index+3], axis=1)
    df_mov_files['subject'] = subject
    
    mov_files_all = mov_files_all.append(df_mov_files[df_mov_files.excel_file!='.DS_Store'])
    

files_emg = np.array(os.listdir(emg_files_path))

emg_files_all = pd.DataFrame()
for folder in files_emg[files_emg!='.DS_Store']:
    df_emg_files = pd.DataFrame({'emg_file': os.listdir(emg_files_path+'/'+folder)})
    df_emg_files = df_emg_files[df_emg_files['emg_file'].str.contains('.vhdr')]
    df_emg_files['session'] = np.where(df_emg_files['emg_file'].str.contains('1|2|3')==True, df_emg_files['emg_file'].str[-6], np.NaN)
    df_emg_files['muscle'] = np.where((df_emg_files['emg_file'].str.contains('1|2|3')==True)
                                      &(df_emg_files['emg_file'].str.contains('APB|ADM')==True), df_emg_files['emg_file'].str[-10:-7], np.NaN)
    
    df_emg_files['muscle_index'] = np.where(df_emg_files.emg_file.str.contains('APB')==True
                                            , df_emg_files['emg_file'].str.find('APB'),
                                            df_emg_files['emg_file'].str.find('ADM'))
    
    df_emg_files['muscle'] = df_emg_files.apply(lambda x: x.emg_file[x.muscle_index:x.muscle_index+3], axis=1)
    
    df_emg_files['subject'] = folder
    emg_files_all = emg_files_all.append(df_emg_files[(df_emg_files.session.isna()==False)])
    
emg_mov_couples = mov_files_all.merge(emg_files_all, how='left', on = ['session','muscle','subject'])

algo_couples = emg_mov_couples[['excel_file','emg_file','subject','muscle','session']]

#### see list of subjects movement Done

In [ ]:
print('Sibjects movement done:', list(algo_couples.subject.unique()))

#### Adjast: Select subjects to run pMA (fill subjects_to_analyse list)

In [ ]:
subjects_to_analyse = ['MedVer_11', 'AnVl_28', 'DaSa_27', 'VaGe_25', 'VaKo_21', 
                       'TimSul_10', 'AlKI_19', 'KrLy_20', 'NiCh_15', 'LaKi_31', 
                       'KoYa_30', 'AlFe_16', 'PoAn_12', 'MiBa_13', 
                       'TiAr_14', 'StSh_24', 'RyEk_29', 'ArtDz_9', 'AlLo_26', 
                       'GlePe_18', 'PoSe_23']
    
subjects_to_analyse2 = [ 'VaKo_21',  'AlKI_19', 'AlFe_16',  'ArtDz_9',
                        'PoSe_23']
    
pd.set_option('display.max_rows', 1000)


selected_couples = algo_couples[(algo_couples.subject.isin(subjects_to_analyse2))&(~algo_couples.excel_file.str.contains('_s'))]
selected_couples

#### See number of data by subjects in folders

In [ ]:
selected_couples.groupby(['subject','muscle']).count().excel_file.to_frame().unstack(1).to_clipboard()

## 1. Run pMA analisys for list of subjects (data filtering, pma find, rigt/left hands analysis, output save to excel file)

#### Check average numbers output during run

In [ ]:
for index, couple in tqdm(selected_couples.iterrows()):
    
    try: 
        
        # DATA LOAD
        subject_path = emg_files_path + '/' + str(couple.subject) + '/' + couple.emg_file
        movement_path = excel_files_path + '/' + str(couple.subject) + '/' + couple.excel_file

        # EMG FILTERS
        raw = mne.io.read_raw_brainvision(subject_path, verbose=False)
        # high pass and low pass filters
        filters_hp_lp = raw.load_data().filter(l_freq=20, h_freq=1000, verbose=False)
        # notch filters
        filters_notch = np.array([50, 100, 150, 200])
        raw_filtered = filters_hp_lp.notch_filter(freqs=filters_notch, verbose=False)
        pma_data = raw_filtered.to_data_frame()


        sheet = pd.read_excel(movement_path)
        sheet['slow'] = np.where((sheet['slow'] == '')|(sheet.slow.isna()==True), sheet['slow'].shift(1), sheet['slow'])
        sheet['bad'] = np.where((sheet['bad'] == '')|(sheet.bad.isna()==True), sheet['bad'].shift(1), sheet['bad'])
        sheet['trial_number'] = (2+ np.array(range(sheet.shape[0])))//2
        sheet = sheet[(sheet.bad!='TRUE')&(sheet.bad!=1)]


        print('Subject:',couple.subject, 'active muscle ' , sheet.muscle.unique()[0], ' session ' , couple.session)

        # Left Hand Data processing
        pma_all = pd.DataFrame()

        PMA_START_POSITIVE = 0.25 
        for muscle in tqdm(['lAPB','lADM','lECU']):
            # culculate preactivation thrs in preactivatuon window based on envelop
            thrss = pma_find_preact_thrs(sheet, pma_data, 1000, muscle)
            # find pma times 
            pma = find_pma(pma_data, muscle, thrss, PMA_START_POSITIVE) 
            # calculate rms or each pma
            rms = rms_pma(pma)
            # save all muscles to one df
            pma_all=pma_all.append(rms)
            print(muscle, 'is done')


        pma_all['pma_latency_ms'] = pma_all.pma_time_start - pma_all.movement_start
        pma_all['session'] = couple.session
        pma_all['file'] = couple.emg_file

        pma_format = round(pma_all[[
           'preactivation_mean_uv',
           'preactivation_max_uv',
           'preactivation_std_uv',
           'preactivation_auc_uv',
           'preactivation_window_rms_mean',
            'preac_thrs','pma_duration_ms', 'pma_latency_ms',
           'pma_mean_uv', 'pma_max_uv', 'pma_std_uv','pma_auc_uv',
           'pma_rms_mean',
            'muscle','Trial in session', 'session', 'file']].set_index(['Trial in session','muscle','session', 'file']).unstack(1),2)

        # Left hand process check
        try:
            pma_left_pivot = round(pma_all.groupby(['muscle']).agg({'pma_duration_ms':'mean',
                                         'pma_latency_ms':'mean',
                                         'pma_rms_mean':'mean',
                                         'pma_mean_uv':'mean',
                                         'pma_wnd':'count', # = how many pma found
                                         'movement_start':'count' # = how many movemnts
                                        }),1)

            print('PMA mean activity for ', couple.subject, 'active muscle ' , sheet.muscle.unique()[0], ' session ' , couple.session)
            print(pma_left_pivot)
        except:
            print('No numeric data for PMA',couple.subject, 'active muscle ' , sheet.muscle.unique()[0], ' session ' , couple.session )
            pass

        # Right Hand processing 
        active_muscle = sheet.muscle.unique()[0]
        r_hand = active_hand(pma_all[pma_all.muscle=='lAPB'][['movement_start', 'movement_end','indexes','Trial in session','Is slow']], pma_data, 10, active_muscle)

        r_hand['session'] = couple.session
        r_hand['file'] = couple.emg_file

        r_hand_format = round(r_hand[['movement_start','movement_end',
           'preactivation_mean_uv',
           'preactivation_max_uv',
           'preactivation_std_uv',
           'preactivation_auc_uv',
           'activation_mean_uv',
           'activation_max_uv',
            'activation_auc_uv',
           'activation_std_uv',
           'muscle','Trial in session','session', 'Is slow','file']].set_index(['Trial in session','muscle','session', 'file']).unstack(1),2)


        # Right hand process check

        try:
            right_hand_pivot = round(r_hand.groupby(['muscle']).agg({'preactivation_mean_uv':'mean',
                                         'activation_mean_uv':'mean',
                                         'activation_auc_uv':'mean',
                                         'movement_start':'count' # = how many movements done
                                        }),1)

            print('Right Hand mean activity for ', couple.subject, 'active muscle ' , sheet.muscle.unique()[0], ' session ' , couple.session)
            print(right_hand_pivot)

        except:
            print('No numeric data for Right hand ',couple.subject, 'active muscle ' , sheet.muscle.unique()[0], ' session ' , couple.session )
            pass

        output = r_hand_format.merge(pma_format, how='left', on = ['Trial in session','file','session'], suffixes = ['_active','_pma'])
        output['active_muscle']  = sheet.muscle.unique()[0]
        output = output.set_index('active_muscle', append=True)


        file_name = 'algo_' + couple.subject + '_' + couple.muscle +'_s' + couple.session + '.xls'

        output.to_excel(file_name)

        print('pma data saved to: ', file_name)
    
    except Exception as er:
        print('Failed ', couple.subject, couple.muscle, couple.session, er)
        pass

### Auto pMA analysis is Done

Mudrich: Latency of pMA was subsequently defined automatically during the delay between burst-onset of FDIVol and the time point, at which muscular activity in the contralateral (resting) FDIpMA exceeded a threshold of its own mean baseline activity (1000 ms pre-FDIVol burst onset) + 2 SD for a time window of at least 10 ms

# Part 3. All 6 files to one for each subject

In [ ]:
import os
pma_analisys_folder_path = '/Users/ksenia.kozlova/Documents/GitHub/pma_grant'
part2_files = np.array(os.listdir(pma_analisys_folder_path))

In [ ]:
needed_files_df = pd.DataFrame(part2_files[pd.Series(part2_files).str.contains('algo')])
needed_files_df['subject'] =  needed_files_df.apply(lambda x: x[0][5:11],axis=1)
needed_files_df['session'] =  needed_files_df.apply(lambda x: x[0][-5],axis=1)
needed_files_df['muscle'] =  needed_files_df.apply(lambda x: x[0][-10:-7],axis=1)
needed_files_df = needed_files_df.sort_values(by=['subject','muscle','session'])
needed_files_df.head(7)

In [ ]:
for subj in needed_files_df.subject.unique():
    all_sessions_file = pd.DataFrame()
    files_subject = needed_files_df[needed_files_df.subject == subj]
    for i, file in files_subject.iterrows():
        sess = pd.read_excel(pma_analisys_folder_path+'/'+file[0], header=[0,1,2])
        all_sessions_file = all_sessions_file.append(sess)
    all_sessions_file.to_excel(subj+'.xls')